# Automatize the generation of an Environment for Grid2op

This notebook will automatize processes to generate an environment for the IEEE 118 grid using *grid2op*. It takes as an input a pre-processed cased and it will generate the respective environment to be used with grid2op.

`case_full_grid_ieee_118 --> | run all notebook | --> subnet ENV grid2op`

The input folder should contain the follwing:
```
|---case_full_grid_ieee_118
    |--- config.py
    |--- grid.json
    |--- grid_layout.json
    |--- loads_charac.csv
    |--- prods_chatac.csv
    |--- params.json
    |--- params_opf.json
```

**Remainder** <br>
**++ ++ ++ +** <br>
 - **It only works for now if you have generated the subgrid case** for grid2op (otherwise go through notebook subgrid_design.ipynb) and preferably for region R2 (not tested for the 2 others

 - **Make sure to have the right slack ids**

 - You need to have **a virtualEnv of Chronix2Grid installed** in Chronix2Grid_folder

 - If you change AWS machine type, you might need to **recompile lightSim2Grid**
 
 - Create virtualenv for Chronix2grid

## Structure and levels:
```
|---main_root
    |---Chronix2grid
    |   |---input_data
    |   |   |---generation
    |   |       |---case_runner_grid (full ieee 118 full grid)
    |   |       |---case_subgrid
    |   |--output_data
    |---Grid2Op_Environment
        |---case_subgrid
    
```

## Processes:

 - Parameters Configuration:
   - number of scenarios
   - number of cores
   - chronix version (for data)
   - subgrid to generate ENV {1, 2, 3}
   - home directory or main_root
   - Slack gen
 - Pre-process input case changing Pmax and Ramps
 
 1) **Chronics Generation For Full IEEE 118** <br>
   *(Run Economic Dispatch using Chronix2grid)* <br>
 2) **Run the runner with the do nothing agent** <br>
   *(Run Grid2op runner from Economic Dispatch results)* <br>
 3) **Correct the slack bus chronic with it** <br>
   *(Run sub-process to compensate slack gen)* <br>
 4) **Create the subgrid chronics** <br>
   *(Using Economic dispatch rel and runner agent, only* <br>
   *specified subnet chronix are created)* <br>
 4) **Run runner in subgrid chronics** <br>
   *(Run Grid2op runner on subgrid chronics)* <br>
 5) **Correct the slack bus chronic** <br>
   *(Correct slack in subgrid chronic)*

## Import libraries

In [2]:
import os
from pprint import pprint
import pandas as pd
import numpy as np
import subprocess
import psutil
from glob import glob

## Parameter Configuration

**--> specify Home directory**

In [3]:
#home_directory="/Users/antoinemarot/dev"#"/Users/camiloromero/Rte/"#/home/ubuntu/
# home_directory = '/Users/camiloromero/Rte/'
home_directory = '/home/ubuntu/'

In [4]:
if not os.path.exists(home_directory):
    print('WARNING: you need to change your home directory '+home_directory+' which does not exist')
    raise

## Input variables

In [5]:
notebook_config = {
    'full_case_name': 'case118_l2rpn_neurips_1x',
    'subgrid_case_name': 'ieee118_R2subgrid_neurips_1x',
    'num_scenarios': 48,
    'num_cores': psutil.cpu_count(logical=True),
    'agent_rel_version': '1x',
    'region': 2, 
    'mode_test': False,
    'withseeds': True,
    'path_read_seeds': '/home/ubuntu/ChroniX2Grid/48years_chronix_v4',
    'chronix_env_name': 'venv_chronix2grid',
    'ref_chonix': '/home/ubuntu/Grid2Op_EnvironmentDesign/case118_l2rpn_neurips_3x'
}

**--> Replace the number of cores and scenarios**

In [6]:
# Replace variables to initialize automated process in notebook
n_scenarios = notebook_config['num_scenarios'] #number of scenarios to generate for each month
nb_core = notebook_config['num_cores'] # number of cores on which to run the computation

**--> Replace the name of your virtual env for chronix2grid**

In [7]:
venv_chronix2grid_name = notebook_config['chronix_env_name']

**--> Replace version data, subregion to generate**

In [8]:
ChronixVersion = notebook_config['agent_rel_version'] #the version of the iteration we are doing for generating our environment
subgrid_region = notebook_config['region']  #choose subgrid region betwen 1,2,3 for IEEE 118 subgrids

#######
ModeTest = notebook_config['mode_test'] #if Test notebook, this should be true, and everything will be run on 2 chronics over 2 months only to also test for multiprocess

#######
ModeRunnerWithSeeds = notebook_config['withseeds'] # Will generate Chronix with specific seed according the path specified.

**--> specify slack id and name for full grid and subgrid** <br>
Slack Ids are necessary for slack correction - make sure they are correct for the full grid and subgrid

In [9]:
#for full grid
idxSlack=37
nameSlack='gen_68_37'

#for subgrid
idxSlack_subgrid=21
nameSlack_subgrid='gen_35_21'#it has changed name in the subgrid

**--> specify paths** <br>

In [10]:
'''Specify Chronix2grid paths'''
Chronix2Grid_folder=os.path.join(home_directory, "ChroniX2Grid")
Input_folder=os.path.join(Chronix2Grid_folder, "input_data") #chronix2grid input folder with necesaary patterns and grid case
Ouput_folder=os.path.join(Chronix2Grid_folder, "output_data") #output folder for chronics

'''Specify Grid2Op_EnvironmentDesign path'''
RunnerScriptFolder=os.path.join(home_directory, "Grid2Op_EnvironmentDesign")

'''Specify full_ieee_118_grid and subgrid NAMES'''
case_runner_grid = notebook_config['full_case_name']
case_subgrid = notebook_config['subgrid_case_name']

'''Specify output folder'''
Ouput_folder_do_nothing=os.path.join(Chronix2Grid_folder, "agent_results" + "_" + ChronixVersion) #output folder when running the runner on chronics

'''Specify path to read seeds'''
seed_read_path = notebook_config['path_read_seeds']

In [11]:
gridPath=os.path.join(Input_folder, 'generation', case_runner_grid)
if not os.path.exists(gridPath):
    print('WARNING: the grid folder '+gridPath+' does not exist, create it')
    raise

In [12]:
venv_chronix2grid_path=os.path.join(Chronix2Grid_folder, venv_chronix2grid_name)
if not os.path.exists(venv_chronix2grid_path):
    print('WARNING: the chronix2grid virtual env folder '+venv_chronix2grid_path+' does not exist, create it')
    raise

In [13]:
# subgridPath=os.path.join(RunnerScriptFolder,case_subgrid)
# if not os.path.exists(subgridPath):
#     print('WARNING: the subgrid folder '+subgridPath+' does not exist, be sure to locate your subgrid folder there.' \
#           'If you have not yet created it, run the subgrid_design.ipynb first')
#     raise

The Process is:
    
    Preliminaries: make sure you have the subgrid case created + create a modified case for the slack to run chroniX2Grid 

    1) Chronics Generation For Full IEEE 118
        - move all chronics in a same folder 
        - duplicate the chronics folder to keep one origal and one that we will modify then
    2) Run the runner with the do nothing agent
        - we want to assess the residual losses that we did not account for in the chronix generation process
    3) Correct the slack bus chronic with it
        - so that no more compensation exists when we will run the chronix again, to avoid compensation making losse management negligeable to the amount of compensation
    4) Create the subgrid chronics
    5) Run the runner with the do nothing agent
        - so that we are all set to analyze the environement and calibrate the thermal limits
        
    6) Last check to see that the slack does not need to compensate and be corrected anymore
     

## Preliminary - create a grid case with lower Pmax and Ramps

<div class="alert alert-block alert-success">
This section may allow you to modify any input file that helps to create an environment in Grid2op and consequentently will affect the Economic Dispatch in the next process.
</div>

This is the case that will be used for Chronix2Grid generation.

This will allow for slack correction later on the real case

In [14]:
!pwd

/Users/antoinemarot/dev/Grid2Op_EnvironmentDesign


In [15]:
case_chronix2grid=case_runner_grid+"_modifySlackBeforeChronixGeneration"

Copy the original case to create the slack modified case

In [16]:
Input_folder_case=os.path.join(Input_folder,"generation",case_runner_grid)
case_chronix2grid_folder=os.path.join(Input_folder,"generation",case_chronix2grid)

if os.path.exists(case_chronix2grid_folder):
    print('delete folder before creating:'+case_chronix2grid_folder)
    !rm -rf $case_chronix2grid_folder
    
!cp -r $Input_folder_case/. $case_chronix2grid_folder

In [17]:
case_chronix2grid_folder

'/Users/antoinemarot/dev/ChroniX2Grid/input_data/generation/case118_l2rpn_neurips_1x_modifySlackBeforeChronixGeneration'

Change the slack values in prods.caract

In [18]:
prod_caract_df=pd.read_csv(os.path.join(case_chronix2grid_folder,'prods_charac.csv'))
prod_caract_df.head()

,name,Pmax,Pmin,type,bus,max_ramp_up,max_ramp_down,min_up_time,min_down_time,marginal_cost,shut_down_cost,start_cost,zone,generator,x,y,V
0,gen_3_0,33.6,0.0,wind,3,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,0,62,-101,142.1
1,gen_3_1,33.6,0.0,wind,3,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,70,62,-101,142.1
2,gen_7_2,400.0,0.0,nuclear,7,2.8,2.8,2016.0,2016.0,35.0,0,27728.7,R1,2,74,-293,355.4
3,gen_9_3,300.0,0.0,thermal,9,5.8,5.8,12.0,12.0,49.0,0,3392.3,R1,3,74,-390,355.4
4,gen_10_4,37.3,0.0,solar,10,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,69,116,-104,142.1


<div class="alert alert-block alert-warning">
<b>WARNING:</b> we suppose that the slack bus has ID 37
</div>

In [19]:
if not (prod_caract_df['name'][idxSlack]==nameSlack):
    print("are you sure this is the slack ID ?")
    raise

prod_caract_df.loc[[37]].head()

,name,Pmax,Pmin,type,bus,max_ramp_up,max_ramp_down,min_up_time,min_down_time,marginal_cost,shut_down_cost,start_cost,zone,generator,x,y,V
37,gen_68_37,350.0,0.0,thermal,68,9.9,9.9,96.0,48.0,40.0,0,27050.6,R2,28,727,-339,142.1


In [20]:
# PmaxReduction=100
PmaxReduction=150
RampReduction=6

prod_caract_df.at[idxSlack,'Pmax']=np.round(prod_caract_df.at[idxSlack,'Pmax']-PmaxReduction,1)
prod_caract_df.at[idxSlack,'max_ramp_up']=np.round(prod_caract_df.at[idxSlack,'max_ramp_up']-RampReduction,1)
prod_caract_df.at[idxSlack,'max_ramp_down']=np.round(prod_caract_df.at[idxSlack,'max_ramp_down']-RampReduction,1)

prod_caract_df.loc[[37]].head()

,name,Pmax,Pmin,type,bus,max_ramp_up,max_ramp_down,min_up_time,min_down_time,marginal_cost,shut_down_cost,start_cost,zone,generator,x,y,V
37,gen_68_37,200.0,0.0,thermal,68,3.9,3.9,96.0,48.0,40.0,0,27050.6,R2,28,727,-339,142.1


In [24]:
#hydro ramp reduction to smoothen the hydro production patterns, especially when you have same cost
RampHydroReductionFactor=2
idxHydro=prod_caract_df[prod_caract_df.type=='hydro'].index

prod_caract_df.loc[idxHydro,'max_ramp_up']=np.round(prod_caract_df.loc[idxHydro,'max_ramp_up']/RampHydroReductionFactor,1)
prod_caract_df.loc[idxHydro,'max_ramp_down']=np.round(prod_caract_df.loc[idxHydro,'max_ramp_down']/RampHydroReductionFactor,1)


In [23]:
list(idxHydro)

[6, 10, 19, 38, 42, 45, 53]

In [25]:
prod_caract_df.loc[idxHydro].head()

,name,Pmax,Pmin,type,bus,max_ramp_up,max_ramp_down,min_up_time,min_down_time,marginal_cost,shut_down_cost,start_cost,zone,generator,x,y,V
6,gen_11_6,250.0,0.0,hydro,11,4.2,4.2,12.0,12.0,38.0,0,0.0,R1,4,191,-82,142.1
10,gen_17_10,250.0,0.0,hydro,17,4.2,4.2,12.0,12.0,39.0,0,0.0,R1,6,317,-250,142.1
19,gen_41_19,250.0,0.0,hydro,41,4.2,4.2,12.0,12.0,36.0,0,0.0,R2,20,700,-46,142.1
38,gen_69_38,250.0,0.0,hydro,69,4.2,4.2,12.0,12.0,37.0,0,0.0,R1,31,582,-346,142.1
42,gen_76_42,250.0,0.0,hydro,76,4.2,4.2,12.0,12.0,37.0,0,0.0,R2,23,718,-518,142.1


In [26]:
prod_caract_df.to_csv(os.path.join(case_chronix2grid_folder,'prods_charac.csv'),index=False)

In [27]:
case_chronix2grid_folder

'/Users/antoinemarot/dev/ChroniX2Grid/input_data/generation/case118_l2rpn_neurips_1x_modifySlackBeforeChronixGeneration'

### Also change grid_path in params_opf.json 

In [21]:
import json 
  
# Opening JSON file 
filePath=os.path.join(case_chronix2grid_folder,'params_opf.json')
f = open(filePath,) 
  
# returns JSON object as  
# a dictionary 
params_json = json.load(f) 
params_json['grid_path']

'input_data/generation/case118_l2rpn_wcci_modifySlackBeforeChronixGeneration_v2/L2RPN_2020_case118_redesigned.json'

In [22]:
# Change losses compensation (this should be below the minimal ratio of losses we will 
#see after running Grid2op on the chronics) 
#Otherwise it will lead to negative slack production
#params_json['reactive_comp'] = 0.5
params_json['losses_pct'] = 0.4

In [23]:
params_json['grid_path']=os.path.join(case_chronix2grid_folder,'grid.json')
f.close()

In [24]:
with open(filePath, 'w') as f:
    json.dump(params_json, f)

In [25]:
#check
f = open(filePath,) 
params_json = json.load(f) 
params_json['grid_path']

'/home/ubuntu/ChroniX2Grid/input_data/generation/case118_l2rpn_neurips_1x_modifySlackBeforeChronixGeneration/grid.json'

In [26]:
params_json['losses_pct']

0.4

## 1) Chronics Generation For Full IEEE 118

<div class="alert alert-block alert-success">
Run Economic dispatch using Chronix2Grid.
</div>

It will create two folders:

```
|---main_root
    |---Chronix2grid
        |---{#scenarios}_years_chronics_{version} (1)
        |---{#scenarios}_years_chronics_{version}_SlackCorrection (2)
```

Dir (1) will contain the results of the Economic Dispatch <br>
Dir (2) will be created temporarily with the same result as dir (1)

In [27]:
# create a unique chronics folder + copy it to later apply slack compensation
folderChronics=os.path.join(Chronix2Grid_folder, str(n_scenarios) + "years_chronix_" + ChronixVersion)
folderChronicsCorrected=folderChronics+"_SlackCorrection"

if os.path.exists(folderChronics):
    print('delete folder before creating')
    !rm -rf $folderChronics
os.makedirs(folderChronics, exist_ok=True)

if os.path.exists(folderChronicsCorrected):
    print('delete folder before creating')
    !rm -rf $folderChronicsCorrected

delete folder before creating
delete folder before creating


In [28]:
os.makedirs(folderChronics, exist_ok=True)
os.makedirs(folderChronicsCorrected, exist_ok=True)

In [29]:
!ls -lsastr $folderChronics

total 8
4 drwxrwxr-x  2 ubuntu ubuntu 4096 Jun  9 20:09 .
4 drwxrwxr-x 81 ubuntu ubuntu 4096 Jun  9 20:09 ..


In [30]:
!ls -lsastr $folderChronicsCorrected

total 8
4 drwxrwxr-x 81 ubuntu ubuntu 4096 Jun  9 20:09 ..
4 drwxrwxr-x  2 ubuntu ubuntu 4096 Jun  9 20:09 .


In [31]:
!pwd

/home/ubuntu/Grid2Op_EnvironmentDesign


In [32]:
months=["january","february","march","april","may","june",
        "july", "august", "september", "october", "november", "december"]
#months=["november","december"]

start_dates=["2012-01-01", "2012-02-01", "2012-03-01", "2012-04-01", "2012-05-01", "2012-06-01",
             "2012-07-01", "2012-08-01", "2012-09-01", "2012-10-01", "2012-11-01", "2012-12-01"]
#start_dates=["2012-11-01","2012-12-01"]

#os.chdir("ChroniX2Grid/chronix2grid")
os.chdir(Chronix2Grid_folder)
#!source venv_chronix2grid/bin/activate 

#### Read seed form path

In [33]:
seed = {}
if ModeRunnerWithSeeds and os.path.exists(seed_read_path):  
    for file in glob(os.path.join(seed_read_path, '*.json')):
        with open(file) as f:
            seed_file = json.load(f)
            month = os.path.basename(file).split('_')[0]
            # Fill seed
            seed[month] = {'loads': seed_file['loads'],
                           'renewables': seed_file['renewables'],
                           'dispatch': seed_file['dispatch'],
                          }
        
pprint(seed, width=1)

{'april': {'dispatch': 1396426012,
           'loads': 1396426012,
           'renewables': 1396426012},
 'august': {'dispatch': 78396797,
            'loads': 78396797,
            'renewables': 78396797},
 'december': {'dispatch': 688239254,
              'loads': 688239254,
              'renewables': 688239254},
 'february': {'dispatch': 936327420,
              'loads': 936327420,
              'renewables': 936327420},
 'january': {'dispatch': 1180859679,
             'loads': 1180859679,
             'renewables': 1180859679},
 'july': {'dispatch': 1609097239,
          'loads': 1609097239,
          'renewables': 1609097239},
 'june': {'dispatch': 935311786,
          'loads': 935311786,
          'renewables': 935311786},
 'march': {'dispatch': 487137112,
           'loads': 487137112,
           'renewables': 487137112},
 'may': {'dispatch': 138882998,
         'loads': 138882998,
         'renewables': 138882998},
 'november': {'dispatch': 1019804599,
              'loads': 

In [34]:
!pwd

/home/ubuntu/ChroniX2Grid


<div class="alert alert-block alert-info">
    Execute command line to run economic dispatch
</div>

<div class="alert alert-block alert-warning">
<b>WARNING:</b> Chronix2grid virtualenv will be activated
</div>

Generate 4 weeks chronics for every month in the Chronix2grid virtual env

In [35]:
if ModeTest:
    print('we are in testing mode for the process of this notebook')
    months=months[0:2]
    start_dates=start_dates[0:2]
    n_scenarios=2 #number of scenarios to generate for each month
    nb_core=4
    
else:
    print('we are in production mode for the process of this notebook, generating lots of chronics')
    
for i in range(len(months)):
    month=months[i]
    print(month)
    start_date=start_dates[i]

    #define your chronix2grid command line with your arguments
    cli_activateVirtualEnv='. '+venv_chronix2grid_path+'/bin/activate'
    
    if ModeRunnerWithSeeds:
        cli_chronix2grid="chronix2grid \
                         --mode LRT --output-folder {} --input-folder {} --ignore-warnings \
                         --weeks 4 --case {} --n_scenarios {} --nb_core {} --scenario_name {} --start-date {} \
                         --seed-for-loads {} --seed-for-res {} --seed-for-dispatch {}".format(
                         Ouput_folder, Input_folder, case_chronix2grid, n_scenarios, nb_core, month, start_date, \
                         seed[month]['loads'], seed[month]['renewables'], seed[month]['dispatch']) 
        
    else:
        cli_chronix2grid="chronix2grid \
                         --mode LRT --output-folder {} --input-folder {} --ignore-warnings \
                         --weeks 4 --case {} --n_scenarios {} --nb_core {} --scenario_name {} --start-date {}".format(
                         Ouput_folder, Input_folder, case_chronix2grid, n_scenarios, nb_core, month, start_date)
        
    #call a subprocess to load your virtual env and excute your cli with it
#     subprocess.call('. venv_chronix2grid/bin/activate;'+cli_chronix2grid, shell=True)
    output=subprocess.call(cli_activateVirtualEnv+';' + cli_chronix2grid, shell=True)

we are in production mode for the process of this notebook, generating lots of chronics
january
february
march
april
may
june
july
august
september
october
november
december


In [36]:
!ls -lsastr $folderChronics

total 8
4 drwxrwxr-x  2 ubuntu ubuntu 4096 Jun  9 20:09 .
4 drwxrwxr-x 81 ubuntu ubuntu 4096 Jun  9 20:09 ..


<code style="background:yellow;color:black">
Move economic dispatch results to Dir (1)
</code>

In [37]:
existing_folderChronics=os.path.join(Ouput_folder, 'generation', case_chronix2grid)#"${Ouput_folder}/generation/${case}/"

existing_subfolderChronics=next(os.walk(existing_folderChronics))[1]
existing_subfolderChronics=[os.path.join(existing_folderChronics,subF) for subF in existing_subfolderChronics]

for d in existing_subfolderChronics:
    #!ls $d/*
    !mv $d/* $folderChronics

In [38]:
!ls -lsastr $folderChronics

total 2384
 4 drwxrwxr-x  81 ubuntu ubuntu  4096 Jun  9 20:09 ..
 4 -rw-rw-r--   1 ubuntu ubuntu    71 Jun  9 20:11 january_seeds_info.json
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_37
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_10
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_29
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_44
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_28
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_41
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_12
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_20
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_18
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_26
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_27
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:16 Scenario_january_42
 4 drwxr

In [39]:
nFolders=n_scenarios*len(months)
if(len(next(os.walk(folderChronics))[1])!=nFolders):
    print("we have a problem in the chronics generation - maybe chronics to grid did not run properly")
    print("do it in the shell to check")
    print("do 1)  . venv_chronix2grid/bin/activate")
    print("this will source chronix2grid")
    print("do 2) ")
    print(cli_chronix2grid)
    raise

<code style="background:yellow;color:black">
Copy temporarily chronics from Dir (1) to Dir (2)
</code>

In [40]:
#!cp -r $folderChronics/* $folderChronicsCorrected
!cp -r $folderChronics/. $folderChronicsCorrected

In [41]:
!ls -lsastr $folderChronicsCorrected

total 2384
 4 drwxrwxr-x  81 ubuntu ubuntu  4096 Jun  9 20:09 ..
 4 -rw-rw-r--   1 ubuntu ubuntu    71 Jun  9 20:59 september_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    71 Jun  9 20:59 october_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    71 Jun  9 20:59 november_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    68 Jun  9 20:59 may_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    68 Jun  9 20:59 march_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    68 Jun  9 20:59 june_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    71 Jun  9 20:59 july_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    71 Jun  9 20:59 january_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    68 Jun  9 20:59 february_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    65 Jun  9 20:59 august_seeds_info.json
 4 -rw-rw-r--   1 ubuntu ubuntu    71 Jun  9 20:59 april_seeds_info.json
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:59 Scenario_january_00
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 20:59

In [42]:
if(len(next(os.walk(folderChronicsCorrected))[1])!=nFolders):
    print("we have a problem in the chronics copy in:"+folderChronicsCorrected)
    raise

### Seed checker

In [91]:
# Path scenarios to compare with different environments
path_env1 = folderChronics
path_env2 = notebook_config['ref_chonix']

if ModeRunnerWithSeeds:
    # Select random scenarios
    rand_month = np.random.choice(months)
    rand_n_scenario = np.random.choice(range(n_scenarios))
    if rand_n_scenario < 10:
        n_scenario = f'0{rand_n_scenario}'
    else:
        n_scenario = str(rand_n_scenario)

    # Create proper scenario name
    rand_scenario_name = f'Scenario_{rand_month}_{rand_n_scenario}'

    factor_renew = 3
    for var in ['load', 'wind', 'solar']:
        chrnx_2 = os.path.join(path_env2, 'chronics')
        df_env1 = pd.read_csv(os.path.join(path_env1, rand_scenario_name, f'{var}_p.csv.bz2'), sep=';')
        df_env2 = pd.read_csv(os.path.join(chrnx_2, rand_scenario_name, f'{var}_p.csv.bz2'), sep=';')
        if var not in ['wind', 'solar']:
            if (df_env1.values[:, 1:] == df_env2.values[:, 1:]).all():
                print(f'{var}_p are the same in both envs')
        else:
            tolr = 1e-01
            arry1 = factor_renew * df_env1.values[:, 1:].astype(float)
            arry2 = df_env2.values[:, 1:].astype(float)
            if np.allclose(arry1, arry2, tolr):
                print(f'{var}_p are the same in both envs')

load_p are the same in both envs
wind_p are the same in both envs


In [ ]:
# # For solar
# df_env1.iloc[:,1:].sum() * factor_renew

In [ ]:
# df_env2.iloc[:,1:].sum()

## 2) Run the runner with the do nothing agent

<div class="alert alert-block alert-success">
Run Grid2op runner with results previously computed with Chronix2grid
</div>

In [92]:
if os.path.exists(Ouput_folder_do_nothing):
    print('delete folder before creating:'+Ouput_folder_do_nothing)
    !rm -rf $Ouput_folder_do_nothing

os.makedirs(Ouput_folder_do_nothing)

<code style="background:yellow;color:black">
Copy the full grid case in the folder whith the script of the runner
</code>

In [93]:
orginalGridFolder=os.path.join(Input_folder,'generation',case_runner_grid)
workingGridFolder=os.path.join(RunnerScriptFolder,case_runner_grid)

if os.path.exists(workingGridFolder):
    print('delete folder before creating case in:'+workingGridFolder)
    !rm -rf $workingGridFolder
!cp -r $orginalGridFolder $RunnerScriptFolder #to copy the content of a into b do "cp -r a/. b


<code style="background:yellow;color:black">
Move chronics from slack correction to chronics dir where runner will point out.
</code>

move the copy of the chronics in the case folder

In [94]:
runnerCaseFolder=os.path.join(RunnerScriptFolder,case_runner_grid)
runner_chronicsFolder=os.path.join(runnerCaseFolder,'chronics')
if os.path.exists(runner_chronicsFolder) & os.path.exists(folderChronicsCorrected):
    print('delete folder before creating')
    !rm -rf $runner_chronicsFolder
    
os.makedirs(runner_chronicsFolder,exist_ok=True)
!mv $folderChronicsCorrected/* $runner_chronicsFolder

<code style="background:yellow;color:black">
Move results from Dir (2) to Grid2Op_Envirorment/subgrid/chronics
</code>

In [95]:
print(os.listdir(runnerCaseFolder))

['params_opf.json', 'chronics', 'grid_layout.json', 'prods_charac.csv', 'loads_charac.csv', 'params.json', 'config.py', 'grid.json']


In [96]:
# rename the grid name as grid.json
fileToRename=[f for f in os.listdir(runnerCaseFolder) if 'l2rpn' in f.casefold()]
if len(fileToRename)!=0:
    oldFileName=os.path.join(runnerCaseFolder,fileToRename[0])
    newFileName=os.path.join(runnerCaseFolder,'grid.json')
    !mv $oldFileName $newFileName

In [97]:
print(runner_chronicsFolder)
!ls -lsastr $Ouput_folder_do_nothing

/home/ubuntu/Grid2Op_EnvironmentDesign/case118_l2rpn_neurips_1x/chronics
total 8
4 drwxrwxr-x 82 ubuntu ubuntu 4096 Jun  9 21:07 ..
4 drwxrwxr-x  2 ubuntu ubuntu 4096 Jun  9 21:07 .


In [98]:
nbEpisodes=len(next(os.walk(runner_chronicsFolder))[1])
if(nbEpisodes!=nFolders):
    print("we have a problem in the chronics folder of the case file on which the runner will run")
    raise

In [99]:
#check that there is no datetime column in loads_p
#ScenarioNames=next(os.walk(runner_chronicsFolder))[1]
#for s in ScenarioNames:
#    pathScen=os.path.join(runner_chronicsFolder,s)
#    df_scen=pd.read_csv(os.path.join(pathScen,'load_p.csv.bz2'),nrows=1)
#    if('datetime' in df_scen.columns):
#        print(s)

<div class="alert alert-block alert-info">
    Execute command line to run the runner with do nothing
</div>

In [100]:
#datetime columns are sometimes introduced into load chronics... weird...

cli_runner_fullGrid="python "+os.path.join(RunnerScriptFolder,'run_DoNothing_grid2op_onChronix.py')+' --grid_path {} --ouput_dir {} --nb_episode {} --nb_core {}'.format(
    runnerCaseFolder,Ouput_folder_do_nothing,nbEpisodes,nb_core) 
    
    #call a subprocess to load your virtual env and excute your cli with it
#subprocess.call(cli, shell=True)
!$cli_runner_fullGrid
 

the case folder is: /home/ubuntu/Grid2Op_EnvironmentDesign/case118_l2rpn_neurips_1x
the output folder will be: /home/ubuntu/ChroniX2Grid/agent_results_1x
the number of cores used is: 48
the number of scenarios we consider is: 576


In [101]:
!ls -lsastr $Ouput_folder_do_nothing

total 2492
 4 drwxrwxr-x  82 ubuntu ubuntu  4096 Jun  9 21:07 ..
52 -rw-rw-r--   1 ubuntu ubuntu 52955 Jun  9 21:08 dict_action_space.json
52 -rw-rw-r--   1 ubuntu ubuntu 52948 Jun  9 21:08 dict_observation_space.json
52 -rw-rw-r--   1 ubuntu ubuntu 52933 Jun  9 21:08 dict_env_modification_space.json
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_06
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_09
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_03
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_04
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_17
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_12
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_05
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_23
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_30
 4 drwxrwxr-x   2 ubuntu ubuntu  4096 Jun  9 21:09 Scenario_april_38
 4 drwxr

In [102]:
ScenariosWithRunner=next(os.walk(Ouput_folder_do_nothing))[1]
ScenariosWIthChronics=next(os.walk(folderChronics))[1]

isMissing=False

if(len(ScenariosWIthChronics)>len(ScenariosWithRunner)):#something went wrong and we are missing scenarios
    ChronicsNoRunnerFolder=folderChronics+"_MissingRunner"
    
    isMissing=True
    if os.path.exists(ChronicsNoRunnerFolder):
        print('delete folder before creating:'+ChronicsNoRunnerFolder)
        !rm -rf $ChronicsNoRunnerFolder
    
    MissingScenariosForRunner=list(set(ScenariosWIthChronics) - set(ScenariosWithRunner))
    
    for scenario in MissingScenariosForRunner:
        print(scenario)
        scenarioPath=os.path.join(folderChronics,scenario)
        #scenarioNewPath=os.path.join(ChronicsNoRunnerFolder,scenario)
        !mv $scenarioPath $ChronicsNoRunnerFolder/
        

In [103]:
ScenariosWithRunner=next(os.walk(Ouput_folder_do_nothing))[1]
ScenariosWIthChronics=next(os.walk(folderChronics))[1]

if(len(ScenariosWIthChronics)>len(ScenariosWithRunner)):#something went wrong and we are missing scenarios
    print('what')
elif isMissing:
    print('we are good now')
    #delete chronics folder in runnerCaseFolder and create a new one
    if os.path.exists(runner_chronicsFolder): #& os.path.exists(folderChronicsCorrected):
        print('delete folder before creating')
        !rm -rf $runner_chronicsFolder
    
    os.makedirs(runner_chronicsFolder)
    !cp -r $folderChronics/. $runner_chronicsFolder
    #os.makedirs(runner_chronicsFolder,exist_ok=True)
    #!mv $folderChronicsCorrected $runner_chronicsFolder

# 3) Correct the slack bus chronic with it

<div class="alert alert-block alert-info">
    Execute command line to correct the slack gen.
</div>

In [104]:
cli_correct_slack_fullGrid="python "+os.path.join(RunnerScriptFolder,'correctForSlack.py')+' --grid_path {} --id_slack {} --slack_name {} --agent_results_path {} --nb_core {}'.format(
    runnerCaseFolder,idxSlack,nameSlack,Ouput_folder_do_nothing,nb_core) 


#!cli_correct_slack #does not work here...


In [105]:
#subprocess.call(['python', 'correctForSlack.py', '--grid_path', runnerCaseFolder,'--id_slack',str(idxSlack),'--slack_name',nameSlack,
#                '--agent_results_path',Ouput_folder_do_nothing,'--nb_core',str(nb_core)])
output=subprocess.check_output(cli_correct_slack_fullGrid, shell=True)

In [106]:
print(output.decode('utf-8'))

multiprocessing done
Time taken = 156.55985641479492 seconds
start correction scenario: Scenario_july_44
maximum compensation for slack before correction in MW: 50.871734619140625
average compensation for slack before correction in MW: 29.569833305209382
median compensation for slack before correction in MW: 29.040611267089844
min compensation for slack before correction in MW: 11.229836463928223
end correction scenario: Scenario_july_44
start correction scenario: Scenario_september_34
maximum compensation for slack before correction in MW: 42.934207916259766
average compensation for slack before correction in MW: 30.173302750159923
median compensation for slack before correction in MW: 30.019697189331055
min compensation for slack before correction in MW: 23.045028686523438
end correction scenario: Scenario_september_34
start correction scenario: Scenario_may_43
maximum compensation for slack before correction in MW: 66.89791107177734
average compensation for slack before correction i

make symlink corrected chronics with Chronix2grid folder

In [107]:

if os.path.exists(folderChronicsCorrected):
    print('delete folder before creating:'+folderChronicsCorrected)
    !rm -rf $folderChronicsCorrected

!ln -s $runner_chronicsFolder $folderChronicsCorrected

delete folder before creating:/home/ubuntu/ChroniX2Grid/48years_chronix_1x_SlackCorrection


#  4) Create the subgrid chronics - Create_chronics_subnet.py

Move subgrid to dir (2)

In [ ]:
FromSubgridPath=os.path.join(Input_folder, 'generation', case_subgrid)
ToSubgridPath=os.path.join(RunnerScriptFolder)

!cp -r $FromSubgridPath $ToSubgridPath #to copy the content of a into b do "cp -r a/. b

check that the subgrid folder exists - or otherwise run the 'subgrid_design' notebook

In [ ]:
subgridPath = os.path.join(RunnerScriptFolder, case_subgrid)
folderChronicsSubgrid=os.path.join(subgridPath, 'chronics')
if os.path.exists(folderChronicsSubgrid):
    print('delete folder before creating')
    !rm -rf $folderChronicsSubgrid

os.makedirs(folderChronicsSubgrid, exist_ok=True)

In [ ]:
!ls -lsastr $folderChronicsSubgrid

In [ ]:
!ls -lsastr $folderChronicsCorrected

<div class="alert alert-block alert-info">
    Execute command line to get chronix for subgrid.
</div>

In [ ]:
cli_create_subgrid_chronics="python "+os.path.join(RunnerScriptFolder,'Create_chronics_subnet.py')+' --subnet_path {} --disptach_dir {} --agent_dir {} --ouput_dir {} --region {} --cores {}'.format(
subgridPath,folderChronicsCorrected,Ouput_folder_do_nothing,folderChronicsSubgrid,subgrid_region,nb_core) 

!$cli_create_subgrid_chronics

In [ ]:
output=subprocess.check_output(cli_create_subgrid_chronics,shell=True)

In [ ]:
print(output.decode('utf-8'))

#  5) Runner on subgrid chronics

In [ ]:
Ouput_folder_do_nothing_subgrid=Ouput_folder_do_nothing+'_subgrid'

if os.path.exists(Ouput_folder_do_nothing_subgrid):
    print('delete folder before creating')
    !rm -rf $Ouput_folder_do_nothing_subgrid

In [ ]:
!ls -lsastr $Ouput_folder_do_nothing_subgrid

In [ ]:
runner_chronicsFolder_subgrid=os.path.join(subgridPath,'chronics')
nbEpisodes=len(next(os.walk(runner_chronicsFolder_subgrid))[1])
nbEpisodes

In [ ]:
print('how to get the subgrid_layout and config.py ? we take it from the full ieee118 by default')
!cp $runnerCaseFolder/grid_layout.json $subgridPath/
!cp $runnerCaseFolder/config.py $subgridPath/

<div class="alert alert-block alert-info">
    Execute command line to run do nothing for subgrid.
</div>

In [ ]:
cli_runner_subgrid="python "+os.path.join(RunnerScriptFolder,'run_DoNothing_grid2op_onChronix.py')+' --grid_path {} --ouput_dir {} --nb_episode {} --nb_core {}'.format(
    subgridPath,Ouput_folder_do_nothing_subgrid,nbEpisodes,nb_core) 
    
    #call a subprocess to load your virtual env and excute your cli with it
#subprocess.call(cli, shell=True)
!$cli_runner_subgrid

In [ ]:
!ls -lsastr $Ouput_folder_do_nothing_subgrid

# 6) Correct the slack bus chronic - check that compensation is now minimal

<div class="alert alert-block alert-warning">
<b>WARNING:</b> Check the id and slack name for subgrid
</div>

In [ ]:
print("idxSlack_subgrid : "+ str(idxSlack_subgrid))
print('nameSlack_subgrid : '+nameSlack_subgrid)#it has changed name in the subgrid

### Compute slack Correction

In [ ]:
cli_correct_slack_subgrid="python "+os.path.join(RunnerScriptFolder,'correctForSlack.py')+' --grid_path {} --id_slack {} --slack_name {} --agent_results_path {} --nb_core {}'.format(
    subgridPath,idxSlack_subgrid,nameSlack_subgrid,Ouput_folder_do_nothing_subgrid,nb_core) 


#!cli_correct_slack #does not work here


In [ ]:
#subprocess.call(['python', 'correctForSlack.py', '--grid_path', runnerCaseFolder,'--id_slack',str(idxSlack),'--slack_name',nameSlack,
#                '--agent_results_path',Ouput_folder_do_nothing,'--nb_core',str(nb_core)])
output=subprocess.check_output(cli_correct_slack_subgrid,shell=True)

In [ ]:
print(output.decode('utf-8'))